Este notebook cria uma base de dados sintética, que aumenta a quantidade de exemplos positivos (que mudaram de emprego), de forma que os algoritmos sejam treinados dando o mesmo grau de importância para os exemplos positivos e negativos, o chamado _balanceamento_.

São criados os arquivos de treino e de teste, sendo que o treino consiste em uma base balanceada, enquanto que o teste contém o percentual de casos positivos e negativos como é encontrado na vida real.

In [1]:
# coding: utf-8

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTEENN

In [3]:
# semente de aleatoriedade
my_seed = 1980

In [4]:
# le arquivo
df = pd.read_csv('../data/df.csv')

In [5]:
# quantidade de valores nao nulos
df.count()

label                         122793
cargo_analista                122793
cargo_consultor               122793
cargo_engenheiro              122793
cargo_estagiario              122793
cargo_gerente                 122793
casado                        122793
curso_administracao           122793
curso_computacao              122793
curso_engenharia              122793
experiencia_meses             122793
homem                         122793
idioma-en-fala                122793
idioma-en-le                  122793
idioma-eo-fala                122793
idioma-es-escreve             122793
idioma-es-fala                122793
idioma-fr-fala                122793
idioma-it-fala                122793
idioma-ja-le                  122793
idioma-nativo-es              122793
idioma-pt-fala                122793
idioma-pt-le                  122793
idioma-ru-fala                122793
loc-br-ba                     122793
loc-br-go                     122793
loc-br-mg                     122793
l

In [6]:
# divide registros entre treino (70%) e teste (30%)
treino, teste = train_test_split(df, test_size=0.3, random_state=my_seed)

In [7]:
# percentual de casos positivos na base de treino
100 * len(treino.loc[treino['label'] == 1]) / float(len(treino))

3.7100808562619974

In [8]:
# percentual de casos positivos na base de teste
100 * len(teste.loc[teste['label'] == 1]) / float(len(teste))

3.933438297410283

In [9]:
# seleciona variaveis preditoras
explicativas = [x for x in treino.columns if x != 'label']

In [10]:
X = treino[explicativas]
y = treino['label']

In [11]:
# aumenta numero de casos positivos usando amostras sinteticas
s = SMOTEENN(random_state=my_seed, ratio="all")
X_s, y_s = s.fit_sample(X, y)

In [12]:
treino_s = pd.concat([pd.Series(y_s, name='label'),
                   pd.DataFrame(X_s, columns=explicativas)], axis=1)

In [13]:
# novo percentual de casos positivos, apos geracao de exemplos sinteticos
100 * treino_s['label'].sum() / float(len(treino_s))

49.824805377179707

In [14]:
treino_s.to_csv('../data/treino.csv', index=False)
teste.to_csv('../data/teste.csv', index=False)